Importando a biblioteca pandas

In [1]:
import pandas as pd

Importando a biblioteca apriori

In [2]:
from mlxtend.frequent_patterns import apriori

Criando o dataset

In [6]:
columns = ['ID', 'Leite', 'Café', 'Cerveja', 'Pão', 'Manteiga', 'Arroz', 'Feijão']
dataset = [
    [1, 0, 1, 0, 1, 1, 0, 0],
    [2, 1, 0, 0, 1, 1, 0, 0],
    [3, 0, 1, 0, 1, 1, 0, 0],
    [4, 1, 1, 0, 1, 1, 0, 0],
    [5, 0, 0, 1, 0, 0, 0, 0],
    [6, 0, 0, 0, 0, 1, 0, 0],
    [7, 0, 0, 0, 1, 0, 0, 0],
    [8, 0, 0, 0, 0, 0, 0, 1],
    [9, 0, 0, 0, 0, 0, 1, 1],
    [10, 0, 0, 0, 0, 0, 1, 0]
]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df = pd.DataFrame(dataset, columns=columns)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Verificando o dataset

In [8]:
print(df)

   ID  Leite  Café  Cerveja  Pão  Manteiga  Arroz  Feijão
0   1      0     1        0    1         1      0       0
1   2      1     0        0    1         1      0       0
2   3      0     1        0    1         1      0       0
3   4      1     1        0    1         1      0       0
4   5      0     0        1    0         0      0       0
5   6      0     0        0    0         1      0       0
6   7      0     0        0    1         0      0       0
7   8      0     0        0    0         0      0       1
8   9      0     0        0    0         0      1       1
9  10      0     0        0    0         0      1       0


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Preparação, construção da classe apriori e definição do threshold = 0.5

In [9]:
class Apriori:
    """Classe Apriori. Contém os passos do algoritmo Apriori."""

    threshold = 0.5
    df = None

    def __init__(self, df, threshold=None, transform_bol=False):
        """Construtor da classe Apriori.

        :param pandas.DataFrame df: dataset de transações (1 ou 0).
        :param float threshold: define o threshold para min_support.
        :return: instância de Apriori.
        :rtype: Apriori
        """
        self._validate_df(df)
        self.df = df

        if threshold is not None:
            self.threshold = threshold

        if transform_bol:
            self._transform_bol()

    def _validate_df(self, df=None):
        """Valida se df existe.

        :param pandas.DataFrame df: dataset de transações (1 ou 0).
        :return:
        :rtype: void
        """
        if df is None:
            raise Exception("df deve ser um pandas.DataFrame válido.")

    def _transform_bol(self):
        """Transforma o dataset (1 ou 0) para (True ou False).

        :return:
        :rtype: void
        """
        for column in self.df.columns:
            self.df[column] = self.df[column].apply(lambda x: True if x == 1 else False)

    def _apriori(self, use_colnames=False, max_len=None, count=True):
        """Chama a função apriori de mlxtend.frequent_patterns.

        :param bool use_colnames: Flag para usar nomes das colunas no DataFrame final.
        :param int max_len: Comprimento máximo dos itemsets gerados.
        :param bool count: Flag para contar o comprimento dos itemsets.
        :return: DataFrame apriori.
        :rtype: pandas.DataFrame
        """
        apriori_df = apriori(
            self.df,
            min_support=self.threshold,
            use_colnames=use_colnames,
            max_len=max_len
        )

        if count:
            apriori_df['length'] = apriori_df['itemsets'].apply(lambda x: len(x))

        return apriori_df

    def run(self, use_colnames=False, max_len=None, count=True):
        """Função executora do Apriori.

        :param bool use_colnames: Flag para usar nomes das colunas no DataFrame final.
        :param int max_len: Comprimento máximo dos itemsets gerados.
        :param bool count: Flag para contar o comprimento dos itemsets.
        :return: DataFrame apriori.
        :rtype: pandas.DataFrame
        """
        return self._apriori(
            use_colnames=use_colnames,
            max_len=max_len,
            count=count
        )

    def filter(self, apriori_df, length, threshold):
        """Filtra o DataFrame Apriori por comprimento e threshold.

        :param pandas.DataFrame apriori_df: DataFrame Apriori.
        :param int length: Comprimento dos itemsets requeridos.
        :param float threshold: Threshold mínimo requerido.
        :return: DataFrame Apriori filtrado.
        :rtype: pandas.DataFrame
        """
        if 'length' not in apriori_df.columns:
            raise Exception("apriori_df não possui length. Execute o Apriori com count=True.")

        return apriori_df[(apriori_df['length'] == length) & (apriori_df['support'] >= threshold)]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Executando o algoritmo apriori

In [10]:
if 'ID' in df.columns:
    del df['ID']  # ID não é relevante para o algoritmo

apriori_runner = Apriori(df, threshold=0.4, transform_bol=True)
apriori_df = apriori_runner.run(use_colnames=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Visualizando os resultados

In [11]:
print(apriori_df)

   support         itemsets  length
0      0.5            (Pão)       1
1      0.5       (Manteiga)       1
2      0.4  (Manteiga, Pão)       2


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


INTERPRETAÇÃO DO RESULTADO

Interpretação dos Resultados:
Vamos interpretar algumas linhas do exemplo acima:

Linha 0:

support: 0.5
itemsets: (Pão)

length: 1

Isso significa que pão aparece em 50% das transações. É um item comum no dataset.

Linha 1:

support: 0.5

itemsets: (Manteiga)

length: 1

Isso significa que manteiga aparece em 50% das transações. É um item comum no dataset.

Linha 2:

support: 0.4

itemsets: (Manteiga, Pão)

length: 2

Isso significa que a combinação manteiga e pão aparecem juntos em 40% das transações. É uma combinação frequente.

SUGESTÕES DE USO
- Pão e manteiga são frequentemente comprados juntos, a loja pode recomendar manteiga aos clientes que compram pão.
- Os produtos podem ser colocados próximos nas prateleiras para aumentar as vendas.
- Ofertas e promoções podem ser planejadas com base em produtos que têm uma alta associação.